# Export EDF

This notebook demonstrates the process of exporting DiveDB data as an EDF file.

While under development, it also contains the prototype (non-library) code; that'll be deleted when this notebook is ready to be merged into the main branch.

Punch list:
- [x] Make a list
- [x] Understand task :) 
- [ ] Prototype:
    - [x] Load basic metadata
    - [x] Load signals
    - [x] Generate EDF file 
        - [X] Can mne serve our needs here? Check if multiple sample rates, arbitrary metadata: edfio can!
        - [x] Decide if different library OR extend mne: use edfio, which is what mne depends on 
    - [x] Test EDF file can be opened externally (e.g. through EDF.jl or other app)
    - [x] Test EDF encodes max/min values
    - [ ] Add metadata to EDF header
- [ ] In tests, write (failing) test for basic new functionality
- [ ] Turn prototype into library code - test passes!
- [ ] Write up edge case tests
    - [ ] Make 'em pass OR file 'em
- [ ] Clean up this notebook (delete this punch list!)
- [ ] Mark PR ready for review

Reminder: this is the end goal

```python
# Example of usage once complete

from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond(os.environ["CONTAINER_DELTA_LAKE_PATH"])

dive_data = duckpond.get_delta_data(    
    labels=["eeg"],
    animal_ids="apfo-001a",
)

dive_data.export_to_edf("path_to_output.edf")
```

### Prototype

In [1]:
# 1. Get metadata
import os
import importlib
import DiveDB.services.duck_pond as dp
importlib.reload(dp)

duckpond = dp.DuckPond(os.environ["CONTAINER_DELTA_LAKE_PATH"])

# Example from the querying_docs notebook
data = duckpond.get_delta_data(    
    labels=["derived_data_depth"],
    animal_ids="apfo-001a",
    frequency=1/60,  # Once a minute
)
display(data)

# Okay, but is there a way to find out what animal_ids, etc, are available?
# Time to go spelunking!
duckpond.get_db_schema()

# ...okay, cool. :) 

,datetime,derived_data_depth
0,2019-11-07 19:50:00+00:00,-1.982753
1,2019-11-07 19:51:00+00:00,-1.900138
2,2019-11-07 19:52:00+00:00,-1.660804
3,2019-11-07 19:53:00+00:00,-1.362733
4,2019-11-07 19:54:00+00:00,-1.074403
...,...,...
173,2019-11-07 22:43:00+00:00,-0.466027
174,2019-11-07 22:44:00+00:00,-0.794010
175,2019-11-07 22:45:00+00:00,-1.119214
176,2019-11-07 22:46:00+00:00,-1.418781


┌──────────┬─────────┬────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│ database │ schema  │            name            │                                                                                                                                                          column_names                                                                                                                                                           │              

In [2]:
# Let's try a sql query as well (also ripped from the querying_docs notebook)
labels_df = duckpond.conn.sql(f"""
    SELECT label
    FROM (
        SELECT DISTINCT label
        FROM DataLake
    )
""").df()
# display(labels_df)

animals_df = duckpond.conn.sql(f"""
    SELECT animal
    FROM (
        SELECT DISTINCT animal
        FROM DataLake
    )
""").df()
# display(animals_df)

signal_df = duckpond.conn.sql(f"""
    SELECT class, label
    FROM (
        SELECT DISTINCT label, class
        FROM DataLake
    )
""").df()
display(signal_df)


,class,label
0,derived_data_corrected_acc,az
1,logger_data_CC-35,light2
2,logger_data_CC-35,gz
3,derived_data_corrected_gyr,gy
4,sensor_data_magnetometer,mz
...,...,...
56,logger_data_CC-35,my
57,logger_data_CC-35,az
58,sensor_data_gyroscope,gz
59,derived_data_calibrated_mag,mz


In [3]:
signal_df.sort_values(by="class")

,class,label
52,derived_data_calibrated_acc,ax
45,derived_data_calibrated_acc,ay
25,derived_data_calibrated_acc,az
59,derived_data_calibrated_mag,mz
49,derived_data_calibrated_mag,mx
...,...,...
4,sensor_data_magnetometer,mz
41,sensor_data_magnetometer,my
39,sensor_data_magnetometer,mx
29,sensor_data_pressure,sensor_data_pressure


In [4]:
# commenting out b/c otherwise this crashes my kernel (if i do other stuff after it)

# # Once more from the other notebook....
# # Get the filtered data
# resampled_data = duckpond.get_delta_data(    
#     animal_ids="apfo-001a",
#     # Resample values to 100 Hz and make sure each signal has the same time intervals
#     frequency=100,
#     # Aggregation of events (think state events - behaviors) type: state (has state and end dates)
#     classes="sensor_data_accelerometer",
# )
# display(resampled_data)
# # Huh. okay, `frequency` triggering a materialization + resample is interesting, not sure 
# # I would have guessed that from the API! I would have guessed that had to do with 
# # the sampling rate of the recording.

# # Okay, so the output of `get_delta_data` with a set frequency returns the signal as a dataframe.

In [5]:

# Is there a way to get the original sample rate? 
unmaterialized_data = duckpond.get_delta_data(    
    animal_ids="apfo-001a",
    # Resample values to 10 Hz and make sure each signal has the same time intervals
    frequency=None,
    # Aggregation of events (think state events - behaviors) type: state (has state and end dates)
    classes="sensor_data_accelerometer",
)
display(unmaterialized_data)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────────────────────┬───────────────┬────────────────┬───────────────┐
│          datetime           │      ax       │       az       │      ay       │
│  timestamp with time zone   │    double     │     double     │    double     │
├─────────────────────────────┼───────────────┼────────────────┼───────────────┤
│ 2019-11-07 19:50:45+00      │ -0.0071826049 │ -10.6901104125 │  0.0263362182 │
│ 2019-11-07 19:50:45.0025+00 │  0.0167594116 │  -10.637437976 │  0.0407014282 │
│ 2019-11-07 19:50:45.005+00  │ -0.0167594116 │ -10.6565915893 │  0.0430956298 │
│ 2019-11-07 19:50:45.0075+00 │  0.0239420166 │ -10.7356002441 │  0.0430956298 │
│ 2019-11-07 19:50:45.01+00   │  0.0023942016 │ -10.6158901611 │  0.0478840332 │
│ 2019-11-07 19:50:45.0125+00 │  0.0023942016 │ -10.7236292358 │  0.0622492431 │
│ 2019-11-07 19:50:45.015+00  │  0.0526724365 │ -10.6637741943 │  0.0287304199 │
│ 2019-11-07 19:50:45.0175+00 │  0.0071826049 │ -10.6565915893 │  0.0454898315 │
│ 2019-11-07 19:50:45.02+00 

In [6]:
# ... okay, got it. now, let's do what needs doing. 
# But also, keep in mind that we should NOT pass a frequency into `get_delta_data`
# before EDF export unless we are very explicit about what we are doing and why. 

# When we don't pass in a frequency (i.e., resample), we get a DuckDBPyRelation
# out of `get_delta_data`
print(type(unmaterialized_data))

# ...from task, I think we want a DuckDBPyConnection instead? Currently unclear to me
# how these interop.

<class 'duckdb.duckdb.DuckDBPyRelation'>


In [139]:
# Okay, now to an EDF! 
# Let's do the demo from edfio (what mne depends on for its EDF support)

from edfio import Edf, EdfSignal, read_edf
import numpy as np
import math

# edfio's example
example_edf = Edf(
    [
        EdfSignal(np.random.randn(30 * 256), sampling_frequency=256, label="EEG Fpz"),
        EdfSignal(np.random.randn(30), sampling_frequency=1, label="Body Temp"),
    ]
)

outpath = ".tmp/example.edf"
example_edf.write(outpath)

example_edf_roundtrip = read_edf(outpath)
display(example_edf_roundtrip.signals)
display(example_edf_roundtrip.signals[0].data)


(<EdfSignal EEG Fpz 256Hz>, <EdfSignal Body Temp 1Hz>)

array([ 0.96627737, -0.16337485,  0.20449128, ...,  0.51771356,
        0.59518333, -2.06714838])

In [ ]:
# ...and now with our data!
# Can we make an EDF from our data? 
# intentionally picking signals with different sampling rates

# ...normally we could query these all at the same time, except that we're putting limits
# on here so that we don't have to get ALL values for each signal. Also, in real case, 
# this is where we'd pull all data and then split it up and make one EDF file per animal/deployment/etc. 
# For now? Hard code it, bebe!
signals = []
classes = ["sensor_data_accelerometer","sensor_data_pressure"]

sig_connections = []
max_duration_sec = 0

# Set up for EDF - first figure out what common max duration is, etc
for class_name in classes:
    df_raw = duckpond.get_delta_data(    
        animal_ids="apfo-001a",
        classes=[class_name],
        # limit=1000,
    )

    #TODO-optimize: surely there is a way to get the number of samples without loading them all?? 
    # If so, do that!
    sampling_rate = df["datetime"].diff()[1:].dt.total_seconds().unique()[0]
    sampling_frequency = int(1/sampling_rate) # TODO-safety: don't just blindly round o_O

    # TODO-correctly! need to figure out max signal length, then start time, then 
    # Lpad to the correct start time + lpad to the correct stop time (lol EDF)
    # For now, we're faking it. We happen to know that the maximum duration signal 
    # of these two is 20 s, so lets zero-pad to that:
    sig_max_duration_sec = math.ceil(df_raw.shape[0] / sampling_frequency)
    max_duration_sec = max(max_duration_sec, sig_max_duration_sec)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# Now actually use the real data to create an edf
for class_name in classes:
    df = duckpond.get_delta_data(    
        animal_ids="apfo-001a",
        classes=[class_name],
        # limit=1000,
    ).df()

    # TODO-safety: check that there's only one value after the unique, check that 
    # this is an integer value or whatever the EDF spec requires, etc
    sampling_rate = df["datetime"].diff()[1:].dt.total_seconds().unique()[0]
    sampling_frequency = int(1/sampling_rate) # TODO-safety: don't just blindly round, see if we allow floating point values here also?? o_O

    signal_label = class_name if len(class_name) <= 16 else class_name[0:16] # lol EDF

    # Need to figure out max signal length, then start time, then 
    # Lpad to the correct start time + lpad to the correct stop time (lol EDF)
    # TODO-future: instead of padding w/ 0, use some signal-specific value
    num_channels = df.shape[1] - 1
    signal_data = np.zeros((max_duration_sec * sampling_frequency, num_channels), dtype=np.float64)
    for i in range(0, num_channels):
        channel_label = (df.columns)[i + 1] #TODO: save channel labels
        num_samples = len(df[channel_label].values)
        signal_data[0:num_samples, i] = df[channel_label].values

    signal = EdfSignal(signal_data,
                        sampling_frequency=sampling_frequency, 
                        label=signal_label)

    signals.append(signal)
    
divedb_edf = Edf(signals)
path = ".tmp/prototype.edf"
divedb_edf.write(path)

divedb_edf_roundtrip = read_edf(path)
display(divedb_edf_roundtrip.signals)
display(divedb_edf_roundtrip.signals[0].data)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

KeyError: 'ax'

In [85]:
class_name = "sensor_data_accelerometer"
df = duckpond.get_delta_data(    
    animal_ids="apfo-001a",
    classes=[class_name],
    # limit=1000,
).df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:

# TODO-safety: check that there's only one value after the unique, check that 
# this is an integer value or whatever the EDF spec requires, etc
sampling_rate = df["datetime"].diff()[1:].dt.total_seconds().unique()[0]
sampling_frequency = int(1/sampling_rate) # TODO-safety: don't just blindly round o_O

label_sanitized = label if len(label) <= 16 else label[0:16] # lol EDF

# TODO-correctly! need to figure out max signal length, then start time, then 
# Lpad to the correct start time + lpad to the correct stop time (lol EDF)
# For now, we're faking it. We happen to know that the maximum duration signal 
# of these two is 20 s, so lets zero-pad to that:
max_duration_sec = math.ceil(df.shape[0] / sampling_frequency) # Find over all input signals
num_channels = df.shape[1] - 1
signal_data = np.zeros((max_duration_sec * sampling_frequency, num_channels), dtype=np.float64)
num_samples = len(df[channel_label].values)
for i in range(0, num_channels):
    channel_label = (df.columns)[i + 1]
    num_samples = len(df[channel_label].values)
    signal_data[0:num_samples, i] = df[channel_label].values

signal = EdfSignal(signal_data,
                    sampling_frequency=sampling_frequency, 
                    label=label_sanitized)

In [9]:
# Commenting out to avoid an unnecessary error! Uncomment to see the (fully expected) error. :) 
# # Can an edf contain nan or inf? 

# sig = np.random.randn(30 * 256)
# sig[0] = np.nan
# print(sig)
# example_edf = Edf([EdfSignal(sig, sampling_frequency=256, label="EEG Fpz")])

# # nope! that answers that.

In [28]:
for sig in divedb_edf_roundtrip.signals:
    print(sig)
    print(sig.__dict__)

sig = divedb_edf_roundtrip.signals[0]

<EdfSignal ax 400Hz>
{'_sampling_frequency': 400.0, '_label': b'ax              ', '_transducer_type': b'                                                                                ', '_physical_dimension': b'        ', '_physical_min': b'0       ', '_physical_max': b'7.657037', '_digital_min': b'-32768  ', '_digital_max': b'32767   ', '_prefiltering': b'                                                                                ', '_samples_per_data_record': b'400     ', '_reserved': b'                                ', '_lazy_loader': None, '_digital': memmap([ 15913,  15753,  15642, ..., -32768, -32768, -32768], dtype=int16)}
<EdfSignal derived_data_dep 50Hz>
{'_sampling_frequency': 50.0, '_label': b'derived_data_dep', '_transducer_type': b'                                                                                ', '_physical_dimension': b'        ', '_physical_min': b'-2.00532', '_physical_max': b'-1.95383', '_digital_min': b'-32768  ', '_digital_max': b'32767   ', '

In [30]:
sig.__dict__

{'_sampling_frequency': 400.0,
 '_label': b'ax              ',
 '_transducer_type': b'                                                                                ',
 '_physical_dimension': b'        ',
 '_physical_min': b'0       ',
 '_physical_max': b'7.657037',
 '_digital_min': b'-32768  ',
 '_digital_max': b'32767   ',
 '_prefiltering': b'                                                                                ',
 '_samples_per_data_record': b'400     ',
 '_reserved': b'                                ',
 '_lazy_loader': None,
 '_digital': memmap([ 15913,  15753,  15642, ..., -32768, -32768, -32768], dtype=int16)}

In [67]:
# Let's practice setting the other fields for the recording 
from edfio import Patient, Recording
import datetime
import json

edf = divedb_edf_roundtrip.copy()

# Okay, looks like these additional fields are VERY strict, disallow spaces, basically can't 
# be json. According to PA, canonical thing to do here is use annotations, so we'll do that 
# for anything interesting. Single world fields/responses? allowed, in the kwarg form 
additional = ('kwarg1', 'value1', 'kwarg2', 'value2')

edf.recording = Recording(
    # startdate=datetime.date(2002, 2, 2), #TODO
    equipment_code="X", #TODO
)

path = ".tmp/prototype.edf"
edf.write(path)
edf.__dict__


{'_version': b'0       ',
 '_local_patient_identification': b'X X X X                                                                         ',
 '_local_recording_identification': b'Startdate X X X X                                                               ',
 '_startdate': b'01.01.85',
 '_starttime': b'00.00.00',
 '_bytes_in_header_record': b'768     ',
 '_reserved': b'                                            ',
 '_num_data_records': b'20      ',
 '_data_record_duration': b'1       ',
 '_num_signals': b'2   ',
 '_signals': (<EdfSignal ax 400Hz>, <EdfSignal derived_data_dep 50Hz>)}

In [65]:
for i in range(1, 30):
    x = edf.recording.get_subfield(i)
    print(i, x)
    if x == 'X':
        pass


1 X
2 X
3 X
4 X
5 blob
6 foo
7 X
8 X
9 X
10 X
11 X
12 X
13 X
14 X
15 X
16 X
17 X
18 X
19 X
20 X
21 X
22 X
23 X
24 X
25 X
26 X
27 X
28 X
29 X


In [32]:
divedb_edf.recording.__dict__

{'_local_recording_identification': 'Startdate X X X X'}

In [10]:
# Huzzah! Time to clean up :) 
# ...actually false. Time to figure out how to get the metadata into the EDF header!
# Check the edfio API: https://github.com/the-siesta-group/edfio?tab=readme-ov-file#usage 
# and https://edfio.readthedocs.io/en/stable/examples.html 